# Tutorial for Handling Multiple Runs of RADLite

**Welcome!**

This tutorial walks through how to generate, combine, and/or compare spectra from across different runs (e.g., for different molecules or different physical models) of RADLite.

***NOTE: This tutorial assumes that the user has already read through the previous tutorials "tutorial_RadliteModel.ipynb" and "tutorial_RadliteSpectrum.ipynb").***

Here are all of the topics that we'll cover:

- Getting started
- Performing multiple runs of RADLite
- Generating a spectrum from across different runs
- Comparing spectra from different models

***Preceding tutorial: "tutorial_RadliteSpectrum.ipynb".***
***Next tutorial: "tutorial_newmodels.ipynb".***

## Getting started

Let's import our package as we usually do:

In [ ]:
#Add path to Python-RADLite package to system
import sys
sys.path.append("../../pyradlite/pyradlite/")

#Import Python-RADLite package
import radlite as RDL

Let's also set up strings for the input file, HITRAN data file, and RADMC output paths:

In [ ]:
inmodfilename = "files_for_tutorials/input_radlite.json" #Input file with RADLite parameters
inspecfilename = "files_for_tutorials/input_spectrum.json" #Input file with RADLite parameters
hitranfilename = "files_for_tutorials/data_hitran.json" #Input file with HITRAN parameters
radmcfilepath = "files_for_tutorials/radmc_outputs/" ##Path to the RADMC output files

## Performing multiple runs of RADLite

We can use $\textit{RDL}$ to loop through/run RADLite for different molecule or model setups.  This is a useful technique for comparing the spectra of different sources, or forming a single spectrum consisting of multiple molecules.  Either way, we only need to do the following for each molecule/model:

1. Edit the base input file (particularly the "run_dir" parameter, which is the name of the folder that will contain the output files).
    - If we want to change the underlying physical model, then we need to change the "inp_path" parameter, which points to the folder containing the output files produced by RADMC.
    - If we want to change the molecule (but maintain the same underlying physical model), then we need to change the molecular parameters as applicable ("molname", "temp_fr", "min_abund", etc.).
2. Initialize a new instance of $\textit{RadliteModel}$ using the updated input file
3. Run the $\textit{run_radlite}$ method

In this tutorial, we'll take a look specifically at a multiple-molecule case (but this approach could certainly be more widely applied).

We're going to use this approach to generate 3 different RADLite runs using the same underlying physical model, containing the molecules C$^{18}$O, ortho-H$_2$O, and para-H$_2$O.

Let's start by looping through the input files.  One way to do this would be to explicitly save 3 different input files, and then loop through them.  We're going to do it another way in just a few blocks, but the code for the explicit approach is still available in the next block.  You're more than welcome to explicitly go in and duplicate the input file provided with this tutorial set ("files_for_tutorial/input_radlite.json"), change the parameters, and then fill in the blanks below:

For this tutorial, however, we're going to edit the base input file in place using the $\textit{json}$ Python package.  We'll save the edited version under a placeholding name, and with each new run we'll override the changes we made before.

First, let's save a list of parameters that we want to change within the base file for each run:

In [ ]:
#List of parameters per run to change from what's already in the base file
numruns = 3 #Total number of runs included
run_dir_list = ["rundir_12CO", "rundir_oH2O", "rundir_pH2O"] #Names for run output directories
molname_list = ["C18O", "H2O", "H2O"] #Molecule names for the runs

#Some molecular traits that we are mostly made up
max_abun_list = [1E-4]*numruns #These are made-up maximum abundances
min_abun_list = [1E-9]*numruns #These are made-up minimum abundances
temp_fr_list = [22, 273.2, 273.2] #[Kelvin]; These are actually somewhat realistic temperatures
whichop_list = ["both", "o", "p"] #Ortho vs. para; only matters for H2O in this case

#We're also changing the values below so that the runs don't take too long to finish
max_Eup_list = [4000]*numruns #Maximum upper energy level allowed
passband_list = [80.0, 30.0, 30.0] #Velocity span over which mol. lines will be calculated

Next, let's loop through the different runs, overriding a placeholding .json file each time (feel free to set $\textit{verbose=False}$ at the top of the block if you don't want the verbal output printed for each model).  The following block of code should take ~2 minutes to run:

In [ ]:
verbose = True #Set this to False if you don't want verbal output printed for each model
#Also set an interesting line range
mu_min = 63.5
mu_max = 64.5

#Import json - allows us to read/process/write .json files
import json

#Read in base input .json file as a dictionary
with open(inmodfilename, 'r') as openfile:
    in_dict = json.load(openfile)

#Globally turn on/off verbal output
in_dict["verbose"]["value"] = verbose
#Globally set an interesting line range
in_dict["min_mu"]["value"] = mu_min
in_dict["max_mu"]["value"] = mu_max

#Set up placeholding .json file
tempinfile = "tempinfile.json"

#Loop through the different runs
mod_list = [None]*numruns #Initialize list to hold each RadliteModel instance
for ii in range(0, numruns):
    #Edit the parameters within the read-in dictionary; replace with values for this run
    in_dict["run_dir"]["value"] = run_dir_list[ii] #Changing the output directory for the run
    in_dict["molname"]["value"] = molname_list[ii] #Changing the molecule
    #
    in_dict["min_abun"]["value"] = min_abun_list[ii] #Changing the minimum abundance
    in_dict["max_abun"]["value"] = max_abun_list[ii] #Changing the maximum abundance
    in_dict["temp_fr"]["value"] = temp_fr_list[ii] #Changing the freeze-out temperature
    in_dict["whichop"]["value"] = whichop_list[ii] #Changing the ortho vs. para criterion
    #
    in_dict["max_Eup"]["value"] = max_Eup_list[ii] #Changing the maximum upper energy allowed
    in_dict["passband"]["value"] = passband_list[ii] #Changing the velocity span per mol. line
    #
    
    #Save the edited dictionary as the placeholding .json file
    with open(tempinfile, 'w') as openfile:
        json.dump(in_dict, openfile)
    
    #Initialize a new instance of the RadliteModel class
    mod_list[ii] = RDL.RadliteModel(infilename=tempinfile, hitranfilename=hitranfilename,
                                radmcfilepath=radmcfilepath)
    
    #Run the run_radlite() method
    mod_list[ii].run_radlite()
    print("The {0}th model has finished!\n\n\n".format(ii))

And that's it for running multiple RADLite sessions!  We now have the output from each run stored in their own neat little folders.

## Generating a spectrum from across different runs

Now that we have output from multiple runs of RADLite, we can combine them by listing the run directories for the $\textit{run_paths}$ parameter within the input spectrum .json file ("input_spectrum.json").  Feel free to go into the file and explicitly change this parameter yourself.  But to really drive the point home about simply editing a base .json file, we will use the same trick we pulled before to update a 'placeholding' .json file that we can then feed into the code, like so:

In [ ]:
#Read in base input .json file as a dictionary
with open(inspecfilename, 'r') as openfile:
    in_dict = json.load(openfile)

#Set up placeholding .json file again
tempinfile = "tempinfile.json"

#Edit the run_paths parameter within the read-in dictionary to point to the new output folders
out_dir_list = [(rdhere+"/outputdir/") for rdhere in run_dir_list] #Tack on "outputdir"
in_dict["run_paths"]["value"] = out_dir_list

#Save the edited dictionary as the placeholding .json file
with open(tempinfile, 'w') as openfile:
    json.dump(in_dict, openfile)

#Initialize an instance of the RadliteSpectrum class
multiSpec = RDL.RadliteSpectrum(infilename=tempinfile)

And now we can generate the full spectrum, as we have done before:

In [ ]:
multiSpec.gen_spec()

And we're done!  Let's take a look at all of the lines we have within this one spectrum:

In [ ]:
molinfo = multiSpec.get_attr("molinfo") #Extracted molecular information

#Extract the sorted indices so that we can print the information in order
import numpy as np #Import numpy so that we can extract the sorted indices
sortinds = np.argsort([molinfo[ii]["wavelength"]
                       for ii in range(0, len(molinfo))]) #Indices for sorting by wavelength

#Print the molecular information for the entire spectrum
print("There are {0} molecular lines within this spectrum.".format(len(molinfo)))
print("Here are all molecules, central wavelengths [microns], and upper energies [K]:")
for ii in sortinds:
    print("{0}: {1:.4f}cm, {2:.4f}K".format(molinfo[ii]["molname"],
                                            (molinfo[ii]["wavelength"]*1E4),
                                            molinfo[ii]["Eup_K"]))

And now let's plot the final spectrum:

In [ ]:
#Plot the spectrum
multiSpec.plot_spec("spectrum", linewidth=2, figsize=(15,7))

## Comparing spectra from different models

We can also compare spectra across different models.  For example, say we want to consider the multi-molecular spectrum we generated previously at different source distances.  We can do so by (1) editing the base input file for $\textit{RadliteSpectrum}$, (2) generating each spectrum within a different $\textit{RadliteSpectrum}$ instance, and then (3) overlapping the plots (as we have done in a previous tutorial), like so:

In [ ]:
#Read in base input .json file as a dictionary
with open(inspecfilename, 'r') as openfile:
    in_dict = json.load(openfile)

#Set up placeholding .json file once again
tempinfile = "tempinfile.json"

#Globally edit the run_paths parameter within the read-in dictionary
out_dir_list = [(rdhere+"/outputdir/") for rdhere in run_dir_list] #Tack on "outputdir"
in_dict["run_paths"]["value"] = out_dir_list

#Set up new distances to consider
dist_list = [25, 50, 100, 200] #All in [pc]
numspecs = len(dist_list) #Number of spectra to compare

In [ ]:
#Loop through the different distances
spec_list = [None]*numspecs #Initialize list to hold instances of RadliteSpectrum
for ii in range(0, numspecs):
    #Change the distance within the read-in dictionary
    in_dict["dist"]["value"] = dist_list[ii]
    
    #Save the edited dictionary as the placeholding .json file
    with open(tempinfile, 'w') as openfile:
        json.dump(in_dict, openfile)

    #Initialize an instance of the RadliteSpectrum class
    spec_list[ii] = RDL.RadliteSpectrum(infilename=tempinfile)
    
    #Generate the current spectrum
    spec_list[ii].gen_spec()

In [ ]:
#Plot the spectra on the same plot (using the dopart=True technique discussed previously)
color_list = ["gray", "turquoise", "purple", "black"] #Colors to use for the lines
linestyle_list = ["-", "--", "-", "--"]
alpha = 0.7 #Measure of translucency
linewidth = 3 #Line thickness
leglabel_list = ["{0:d}pc".format(dist_list[ii]) for ii in range(0, numspecs)] #Legend labels

#First set up a base figure
import matplotlib.pyplot as plt #We need this import so that we can create a base figure
fig = plt.figure(figsize=(10, 10)) #The base figure

#Iterate through the spectra (NOT including the very last one)
for ii in range(0, numspecs-1):
    #Place the full spectrum on the plot
    spec_list[ii].plot_spec("spectrum",
                     fig=fig, #The code will add this line to the base figure
                     dopart=True, #This tells the code that this is an incomplete figure
                     linecolor=color_list[ii], linewidth=linewidth,
                     linestyle=linestyle_list[ii], alpha=alpha,
                     leglabel=leglabel_list[ii]) #Label for the legend

#Place the very last spectrum on the plot and also label/display the overall plot
spec_list[-1].plot_spec("spectrum",
                 fig=fig, #The code will add this line to the base figure
                 ylog=True, #Make the y-axis log-scale
                 dopart=False, #This tells the code that this is now a complete figure
                 linecolor=color_list[-1], linewidth=linewidth,
                 linestyle=linestyle_list[-1], alpha=alpha,
                 dolegend=True, legloc="upper right", #Place a legend for the figure
                 leglabel=leglabel_list[-1], #Label for the legend
                 axisfontsize=20, titlefontsize=22,
                 tickfontsize=18, legfontsize=20, #Update the fontsizes
                 title="An Informative Comparison Plot") #Title for the plot

This approach also certainly has multiple applications.  Have fun exploring them!

In the **next tutorial ("tutorial_newmodels.ipynb")**, we'll demonstrate how to use the RADLiteModel class as a base for other phsyical RADLite models.